$ \nabla c_{01} = v$

$ \nabla c_{0} = -\alpha_0 v - c_{01} e_0$

$ \nabla c_{1} = -\alpha_1 v - c_{01} e_1$

where   $\nabla = \left[\partial / \partial k,\ \partial / \partial \alpha_0,\ \partial / \partial \alpha_1 \right],$

$ v = \left( \frac{1}{c_{01}} + \frac{\alpha_0^2}{c_0} + \frac{\alpha_1^2}{c_1} \right)^{-1} \left[ \frac{1}{k},\quad \ln c_0 - \frac{\alpha_0 c_{01}}{c_0}, \quad \ln c_1 - \frac{\alpha_1 c_{01}}{c_1} \right], \qquad e_0 = \left[0,\ 1,\ 0 \right], \quad e_1 = \left[0,\ 0,\ 1\right] $


$\nabla f_{01} = \frac{1}{\sum c_{01}^2} \left[ \sum f v - f_0 \sum c_0 v - f_1 \sum c_1 v + \left( \alpha_0 f_0 + \alpha_1 f_1 - 2 f_{01} \right) \sum c_{01} v \right] + f_0 e_0 + f_1 e_1$

$f'[0,1,2] = v \left( f_{01} - f_0 - f_1 \right) - f_0 c_{01} e_0 - f_1 c_{01} e_1$

$f'[3,4,5] = [c_0, c_1, c_{01}]$

In [ ]:
def Predict(k, a0, a1, n, f0, f1, f01):
    """
    k, a0, a1 : scalars or arrays of shape (N, M)
    n : scalar or array of shape (N)
    f0, f1, f01 : arrays of shape (M)
    
    returns : array of shape (M) or (N, M)
    """
    f0, f1, f01 = np.atleast_1d(f0, f1, f01)
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    ### sahes are either (N, M), or (1, M)
    
    if c01.shape[0] == 1:
        f = c0[0, :]*f0
        f += c1[0, :]*f1
        f += c01[0, :]*f01
    else:
        f = c0*f0[None, :]
        f += c1*f1[None, :]
        f += c01*f01[None, :]
    
    return f


In [ ]:
def Estimate_Liquid_k_a0_a1_I(n, f0, f1, f, Estimation_History, d_L):
    """
    n : array of shape (N)
    f0, f1 : arrays of shape (M)
    f : array of shape (N, M)
    d_L : int
    
    returns : array of resulrts that includes (k, a0, a1)
              arrays of shape (M + 1 - d_L)
    """    
    Results = []
    for i in tqdm(range(len(f0) + 1 - d_L)):
        x0 = np.ones(3)
        res = least_squares(Liquid_fun_I, x0,
                            args=(n, f0[i:i+d_L], f1[i:i+d_L], f[:, i:i+d_L]),
                            bounds=(0, np.inf))
        
        Results.append(res)
    
    
    Estimation = [Results, d_L]
    Estimation_History.append(Estimation)
    
    return Results

In [1]:
def Liquid_df01_I(k, a0, a1, c0, c1, c01, f0, f1, f):
    """
    k, a0, a1 : scalars
    c0, c1, c01 : arrays of shape (N, 1)
    f0, f1 : scalars or arrays of shape (M)
    f : array of shape (N) or (N, M)
    
    returns : array of shape (3) or (M, 3)
    """
    f0, f1 = np.atleast_1d(f0, f1)
    if f.ndim == 1:
        f = f[:,None]
        
    if (c01**2).sum() == 0:
        f01 = np.zeros((3, f0.shape[0]))
    
    else:
        f01 = Liquid_f01(c0, c1, c01, f0, f1, f)
        
        df01 = np.empty((3, f0.shape[0]))
        df01[0] = (a0*f0 + a1*f1 - f01)/k
        
        with np.errstate(divide='ignore', invalid='ignore'):
            c01_ln_c0 = c01 * np.log(c0)
            c01_ln_c1 = c01 * np.log(c1)
            c01_ln_c0[c0==0] = 0 if a0>0 else -np.inf
            c01_ln_c1[c1==0] = 0 if a1>0 else -np.inf
            
        df01[1] = (f * c01_ln_c0[:,0,None]).sum(axis=0)
        df01[2] = (f * c01_ln_c1[:,0,None]).sum(axis=0)
        df01[1] -= f0 * (c0 * c01_ln_c0).sum()
        df01[2] -= f0 * (c0 * c01_ln_c1).sum()
        df01[1] -= f1 * (c1 * c01_ln_c0).sum()
        df01[2] -= f1 * (c1 * c01_ln_c1).sum()
        df01[1] += (a0*f0 + a1*f1 - 2*f01) * (c01 * c01_ln_c0).sum()
        df01[2] += (a0*f0 + a1*f1 - 2*f01) * (c01 * c01_ln_c1).sum()
        df01[1] -= f0
        df01[2] -= f1
        
    return df01.T


In [3]:
def Liquid_df01_II(k, a0, a1, c0, c1, c01, f0, f1, f):
    """
    k, a0, a1 : scalars
    c0, c1, c01 : arrays of shape (N, 1)
    f0, f1 : scalars or arrays of shape (M)
    f : array of shape (N) or (N, M)
    
    returns : scalar or array of shape (M)
    """
    f0, f1 = np.atleast_1d(f0, f1)
    if f.ndim == 1:
        f = f[:,None]
        
    if (c01**2).sum() == 0:
        df01 = np.zeros((f0.shape[0], 3))
    
    else:
        f01 = Liquid_f01(c0, c1, c01, f0, f1, f)
        df01 = (a0*f0 + a1*f1 - f01)/k
        
    return df01


In [5]:
def Liquid_dfun_I(x, n, f0, f1, f):
    """
    x : array of shape (3)
    n : array of shape (N)
    f0, f1 : arrays of shape (d_L)
    f : array of shape (N, d_L)
    
    return : array of shape (N * d_L, 3)
    """
    k, a0, a1 = x[0], x[1], x[2]
    
    if a0 < 0:
        a0 = 0
    if a1 < 0:
        a1 = 0
    
    c0, c1, c01 = Concentrations(k, a0, a1, n)
    f01 = Liquid_f01(c0, c1, c01, f0, f1, f)
    df01 = Liquid_df01_I(k, a0, a1, c0, c1, c01, f0, f1, f)
    

In [6]:
def Find_k_a0_a1_f01(k, a0, a1, n, f0, f1, f, Finding_History, d_L, cycles_1=15, cycles_2=90):
    """
    k, a0, a1 : arrays of shape (M + 1 - d_L) 
    n : array of shape (N)
    f0, f1 : arrays of shape (M)
    f : array of shape (N, M)
    d_L : int
    x0 : array of shape (M + 1 - d_L)
    
    returns : k, a0, a1, f01 : arrays of shape (M + 1 - d_L)
    """
    
    nodes = np.array([[a0.astype('int32'),   a1.astype('int32')  ],
                      [a0.astype('int32')+1, a1.astype('int32')  ],
                      [a0.astype('int32'),   a1.astype('int32')+1],
                      [a0.astype('int32')+1, a1.astype('int32')+1]])
    
    x0 = np.array(k)
    
    cycles_1 +=1
    cycles_2 +=1
    
    k = []
    a0 = []
    a1 = []
    f01 = []
    
    for i in tqdm(range(len(f0) + 1 - d_L)):
        loss = []
        for j in range(4):
            res = least_squares(PEG_fun_discret, x0[i],
                                args=(nodes[j, 0, i], nodes[j, 1, i], n, f0[i:i+d_L], f1[i:i+d_L], f[:, i:i+d_L]),
                                bounds=(0, np.inf), max_nfev=cycles_1)
            
            f01_est = Liquid_f01(res.x, nodes[j, 0, i], nodes[j, 1, i], n,
                                           f0[i:i+d_L], f1[i:i+d_L], f[:, i:i+d_L])
            
            loss.append(Loss(res.x, nodes[j, 0, i], nodes[j, 1, i], n, f0[i:i+d_L], f1[i:i+d_L], f01_est, f[:, i:i+d_L]))
            
        index = np.argmin(loss)
        a0.append(nodes[index, 0, i])
        a1.append(nodes[index, 1, i])
        
        x0i = res.x[0]
        
        res = least_squares(PEG_fun_discret, x0i,
                            args=(a0[i], a1[i], n, f0[i:i+d_L], f1[i:i+d_L], f[:, i:i+d_L]),
                            bounds=(0, np.inf), max_nfev=cycles_2)
        k.append(res.x[0])
        
        f01.append(Liquid_f01(k[i], a0[i], a1[i], n,
                                        f0[i:i+d_L], f1[i:i+d_L], f[:, i:i+d_L])[0])
        
    a0 = np.array(a0)
    a1 = np.array(a1)
    f01 = np.array(f01)
        
    finding = [(k, a0, a1, f01), d_L]
    Finding_History.append(finding)
        
    return k, a0, a1, f01


def PEG_fun_discret(x, a0, a1, n, f0, f1, f):
    """
    x : scalar
    n : array of shape (N)
    f0, f1 : arrays of shape (d_L)
    f : array of shape (N, d_L)
    
    return : array of shape (N * d_L)
    """
    k = x
    
    if a0 < 0:
        a0 = 0
    if a1 < 0:
        a1 = 0
    f01 = Liquid_f01(k, a0, a1, n, f0, f1, f)
    f_pred = Predict(k, a0, a1, n, f0, f1, f01)
    
    return abs(f_pred - f).flatten()

In [ ]:
diff_PEG_New = []
for i in tqdm(range(len(k_corrected))):
    diff_PEG_New.append(Predict(k_corrected[i], a0_corrected[i], a1_corrected[i],
                                n_PEG, PEG_Pure[i+50], NaPB[i+50], f01_corrected[i]))
diff_PEG_New = (np.array(diff_PEG_New)).squeezee().T

diff_PEG_New = (diff_PEG_New - PEG[:, 50:-49])**2
diff_PEG_New = diff_PEG_New.sum(axis=0)

In [ ]:
plt.figure(dpi=100, figsize=(16,9))
plt.minorticks_on()
plt.grid(True, which='both', axis='x')
plt.grid(True, which='major', axis='y')
plt.xlim([400, 5000])
plt.ylim([0, 1])
plt.xlabel('$\lambda$, cm$^{-1}$')

plt.plot(Lambda[50:-49], NaPB[50:-49], label='water')
plt.plot(Lambda[50:-49], PEG_Pure[50:-49], label='PEG')
plt.plot(Lambda[50:-49], diff_PEG_New, label='deviation from model')

plt.legend()
plt.savefig('Graphs/deviation from model')